In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [7, 8, 11]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [7]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.453675746917725, Final Batch Loss: 2.2290611267089844
Epoch 2, Loss: 4.44475793838501, Final Batch Loss: 2.231189250946045
Epoch 3, Loss: 4.4294633865356445, Final Batch Loss: 2.2035324573516846
Epoch 4, Loss: 4.413354158401489, Final Batch Loss: 2.192084550857544
Epoch 5, Loss: 4.423693656921387, Final Batch Loss: 2.220151662826538
Epoch 6, Loss: 4.409757852554321, Final Batch Loss: 2.1953811645507812
Epoch 7, Loss: 4.4122772216796875, Final Batch Loss: 2.2103021144866943
Epoch 8, Loss: 4.400604486465454, Final Batch Loss: 2.196280002593994
Epoch 9, Loss: 4.400053262710571, Final Batch Loss: 2.204777240753174
Epoch 10, Loss: 4.400651216506958, Final Batch Loss: 2.209001302719116
Epoch 11, Loss: 4.387958526611328, Final Batch Loss: 2.188906192779541
Epoch 12, Loss: 4.364265441894531, Final Batch Loss: 2.167839527130127
Epoch 13, Loss: 4.363017559051514, Final Batch Loss: 2.17270827293396
Epoch 14, Loss: 4.373602390289307, Final Batch Loss: 2.196155071258545
Epoch 15, L

Epoch 128, Loss: 2.1222002506256104, Final Batch Loss: 1.0191209316253662
Epoch 129, Loss: 2.060320496559143, Final Batch Loss: 1.0155359506607056
Epoch 130, Loss: 2.2010324001312256, Final Batch Loss: 1.0974479913711548
Epoch 131, Loss: 2.0849387645721436, Final Batch Loss: 1.0813450813293457
Epoch 132, Loss: 2.0368192195892334, Final Batch Loss: 0.9917807579040527
Epoch 133, Loss: 2.044560432434082, Final Batch Loss: 1.0195772647857666
Epoch 134, Loss: 2.1002601385116577, Final Batch Loss: 1.1065977811813354
Epoch 135, Loss: 2.068902552127838, Final Batch Loss: 1.1425652503967285
Epoch 136, Loss: 2.067120313644409, Final Batch Loss: 1.0306068658828735
Epoch 137, Loss: 2.071666419506073, Final Batch Loss: 1.0812193155288696
Epoch 138, Loss: 1.9863812923431396, Final Batch Loss: 1.0020908117294312
Epoch 139, Loss: 2.081805109977722, Final Batch Loss: 1.0539032220840454
Epoch 140, Loss: 1.8822236061096191, Final Batch Loss: 0.9355401396751404
Epoch 141, Loss: 1.9166374206542969, Final B

Epoch 255, Loss: 1.2742061614990234, Final Batch Loss: 0.6134803295135498
Epoch 256, Loss: 1.2300794124603271, Final Batch Loss: 0.6623453497886658
Epoch 257, Loss: 1.1163482069969177, Final Batch Loss: 0.49763351678848267
Epoch 258, Loss: 1.2009881734848022, Final Batch Loss: 0.6269548535346985
Epoch 259, Loss: 1.1169044375419617, Final Batch Loss: 0.5743025541305542
Epoch 260, Loss: 1.202799677848816, Final Batch Loss: 0.5888945460319519
Epoch 261, Loss: 1.047234058380127, Final Batch Loss: 0.5517752766609192
Epoch 262, Loss: 1.232102632522583, Final Batch Loss: 0.7039804458618164
Epoch 263, Loss: 1.2471774816513062, Final Batch Loss: 0.6308202147483826
Epoch 264, Loss: 1.163964033126831, Final Batch Loss: 0.5617830157279968
Epoch 265, Loss: 1.1246622204780579, Final Batch Loss: 0.5506120920181274
Epoch 266, Loss: 1.1127955913543701, Final Batch Loss: 0.5591451525688171
Epoch 267, Loss: 1.1168991923332214, Final Batch Loss: 0.5327377915382385
Epoch 268, Loss: 1.16733717918396, Final 

Epoch 373, Loss: 0.8672861158847809, Final Batch Loss: 0.4369940757751465
Epoch 374, Loss: 0.867527037858963, Final Batch Loss: 0.44675007462501526
Epoch 375, Loss: 0.9040864408016205, Final Batch Loss: 0.4705889821052551
Epoch 376, Loss: 0.8403051495552063, Final Batch Loss: 0.40746381878852844
Epoch 377, Loss: 0.8913387954235077, Final Batch Loss: 0.41260939836502075
Epoch 378, Loss: 0.811932384967804, Final Batch Loss: 0.45001277327537537
Epoch 379, Loss: 0.7794253528118134, Final Batch Loss: 0.35423967242240906
Epoch 380, Loss: 0.991836667060852, Final Batch Loss: 0.5301458239555359
Epoch 381, Loss: 0.7944869697093964, Final Batch Loss: 0.37589308619499207
Epoch 382, Loss: 0.8062061071395874, Final Batch Loss: 0.40479564666748047
Epoch 383, Loss: 0.8711641728878021, Final Batch Loss: 0.4382861256599426
Epoch 384, Loss: 0.9456667900085449, Final Batch Loss: 0.53939288854599
Epoch 385, Loss: 0.7964400053024292, Final Batch Loss: 0.3627665638923645
Epoch 386, Loss: 0.8713759779930115,

Epoch 485, Loss: 0.6512385606765747, Final Batch Loss: 0.2848978638648987
Epoch 486, Loss: 0.7787931263446808, Final Batch Loss: 0.39783909916877747
Epoch 487, Loss: 0.6444219946861267, Final Batch Loss: 0.30178308486938477
Epoch 488, Loss: 0.7199484407901764, Final Batch Loss: 0.3222852945327759
Epoch 489, Loss: 0.7481369078159332, Final Batch Loss: 0.3423008322715759
Epoch 490, Loss: 0.7403354942798615, Final Batch Loss: 0.39439961314201355
Epoch 491, Loss: 0.7590352892875671, Final Batch Loss: 0.38454434275627136
Epoch 492, Loss: 0.7711102962493896, Final Batch Loss: 0.39386245608329773
Epoch 493, Loss: 0.8066699802875519, Final Batch Loss: 0.47095805406570435
Epoch 494, Loss: 0.8014414012432098, Final Batch Loss: 0.44441935420036316
Epoch 495, Loss: 0.6557479202747345, Final Batch Loss: 0.32426509261131287
Epoch 496, Loss: 0.7333805859088898, Final Batch Loss: 0.3305043578147888
Epoch 497, Loss: 0.780603438615799, Final Batch Loss: 0.4651423692703247
Epoch 498, Loss: 0.769289135932

...

Epoch 4465, Loss: 0.3036854416131973, Final Batch Loss: 0.14729498326778412
Epoch 4466, Loss: 0.31619662046432495, Final Batch Loss: 0.20617344975471497
Epoch 4467, Loss: 0.2650189697742462, Final Batch Loss: 0.13976790010929108
Epoch 4468, Loss: 0.2687665820121765, Final Batch Loss: 0.1657356321811676
Epoch 4469, Loss: 0.2519225627183914, Final Batch Loss: 0.09347225725650787
Epoch 4470, Loss: 0.265966571867466, Final Batch Loss: 0.15185140073299408
Epoch 4471, Loss: 0.3160702586174011, Final Batch Loss: 0.1668546348810196
Epoch 4472, Loss: 0.30579744279384613, Final Batch Loss: 0.184715136885643
Epoch 4473, Loss: 0.343270018696785, Final Batch Loss: 0.15449070930480957
Epoch 4474, Loss: 0.3865080028772354, Final Batch Loss: 0.2804645001888275
Epoch 4475, Loss: 0.26053017377853394, Final Batch Loss: 0.13303826749324799
Epoch 4476, Loss: 0.252087764441967, Final Batch Loss: 0.13699571788311005
Epoch 4477, Loss: 0.24934885650873184, Final Batch Loss: 0.09838464111089706
Epoch 4478, Loss

Epoch 4580, Loss: 0.2488253116607666, Final Batch Loss: 0.13110849261283875
Epoch 4581, Loss: 0.3056981563568115, Final Batch Loss: 0.15353621542453766
Epoch 4582, Loss: 0.27515583485364914, Final Batch Loss: 0.1585475355386734
Epoch 4583, Loss: 0.30796270072460175, Final Batch Loss: 0.150270476937294
Epoch 4584, Loss: 0.26827461272478104, Final Batch Loss: 0.09284292906522751
Epoch 4585, Loss: 0.30955033004283905, Final Batch Loss: 0.19780753552913666
Epoch 4586, Loss: 0.286111518740654, Final Batch Loss: 0.16488958895206451
Epoch 4587, Loss: 0.25190701335668564, Final Batch Loss: 0.11712674051523209
Epoch 4588, Loss: 0.332268163561821, Final Batch Loss: 0.20647896826267242
Epoch 4589, Loss: 0.2372891530394554, Final Batch Loss: 0.11375948786735535
Epoch 4590, Loss: 0.28082364797592163, Final Batch Loss: 0.15474675595760345
Epoch 4591, Loss: 0.2467406913638115, Final Batch Loss: 0.11529097706079483
Epoch 4592, Loss: 0.1888696774840355, Final Batch Loss: 0.06256792694330215
Epoch 4593,

Epoch 4706, Loss: 0.2569599449634552, Final Batch Loss: 0.14824533462524414
Epoch 4707, Loss: 0.44254225492477417, Final Batch Loss: 0.296384334564209
Epoch 4708, Loss: 0.2935982793569565, Final Batch Loss: 0.17150817811489105
Epoch 4709, Loss: 0.25239717215299606, Final Batch Loss: 0.13895338773727417
Epoch 4710, Loss: 0.25146201252937317, Final Batch Loss: 0.13678297400474548
Epoch 4711, Loss: 0.2834591791033745, Final Batch Loss: 0.16213278472423553
Epoch 4712, Loss: 0.4144556373357773, Final Batch Loss: 0.2576914429664612
Epoch 4713, Loss: 0.28008227050304413, Final Batch Loss: 0.1112731546163559
Epoch 4714, Loss: 0.2972254827618599, Final Batch Loss: 0.186102956533432
Epoch 4715, Loss: 0.22724303603172302, Final Batch Loss: 0.08082562685012817
Epoch 4716, Loss: 0.2486845999956131, Final Batch Loss: 0.120203897356987
Epoch 4717, Loss: 0.22296156734228134, Final Batch Loss: 0.12303782254457474
Epoch 4718, Loss: 0.25905589014291763, Final Batch Loss: 0.13883382081985474
Epoch 4719, L

Epoch 4821, Loss: 0.21839639544487, Final Batch Loss: 0.09876582026481628
Epoch 4822, Loss: 0.21609608829021454, Final Batch Loss: 0.08790947496891022
Epoch 4823, Loss: 0.2836606800556183, Final Batch Loss: 0.15678194165229797
Epoch 4824, Loss: 0.23359280824661255, Final Batch Loss: 0.11408348381519318
Epoch 4825, Loss: 0.22158069908618927, Final Batch Loss: 0.09210249781608582
Epoch 4826, Loss: 0.24170396476984024, Final Batch Loss: 0.11422818154096603
Epoch 4827, Loss: 0.2566480189561844, Final Batch Loss: 0.14469562470912933
Epoch 4828, Loss: 0.24576938897371292, Final Batch Loss: 0.1450120061635971
Epoch 4829, Loss: 0.21868564188480377, Final Batch Loss: 0.09510572999715805
Epoch 4830, Loss: 0.22230274975299835, Final Batch Loss: 0.1179710328578949
Epoch 4831, Loss: 0.3003305569291115, Final Batch Loss: 0.10900267213582993
Epoch 4832, Loss: 0.4360816329717636, Final Batch Loss: 0.29498839378356934
Epoch 4833, Loss: 0.29116111993789673, Final Batch Loss: 0.13975882530212402
Epoch 48

Epoch 4944, Loss: 0.2510925158858299, Final Batch Loss: 0.10000302642583847
Epoch 4945, Loss: 0.2279646322131157, Final Batch Loss: 0.11211151629686356
Epoch 4946, Loss: 0.3250076621770859, Final Batch Loss: 0.1890028417110443
Epoch 4947, Loss: 0.292865589261055, Final Batch Loss: 0.14968234300613403
Epoch 4948, Loss: 0.24143611639738083, Final Batch Loss: 0.1362287998199463
Epoch 4949, Loss: 0.21271295100450516, Final Batch Loss: 0.09651457518339157
Epoch 4950, Loss: 0.4058024659752846, Final Batch Loss: 0.2847570776939392
Epoch 4951, Loss: 0.24725830554962158, Final Batch Loss: 0.11799374222755432
Epoch 4952, Loss: 0.28911449015140533, Final Batch Loss: 0.16255240142345428
Epoch 4953, Loss: 0.24988631904125214, Final Batch Loss: 0.14566205441951752
Epoch 4954, Loss: 0.2854986637830734, Final Batch Loss: 0.15534867346286774
Epoch 4955, Loss: 0.31068871915340424, Final Batch Loss: 0.1865766942501068
Epoch 4956, Loss: 0.27199646830558777, Final Batch Loss: 0.1523510217666626
Epoch 4957,

In [8]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[ 8  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0]
 [ 0  0  0 11  0  0  0  0  0]
 [ 0  2  0  0  3  0  0  0  0]
 [ 0  0  1  0  0  7  0  0  5]
 [ 0  0  0  0  0  0 15  0  0]
 [ 0  0  0  0  0  0  0  9  0]
 [ 0  0  0  0  0  0  0  0  6]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000         8
           1    0.87500   1.00000   0.93333        14
           2    0.88889   1.00000   0.94118         8
           3    1.00000   1.00000   1.00000        11
           4    1.00000   0.60000   0.75000         5
           5    1.00000   0.53846   0.70000        13
           6    1.00000   1.00000   1.00000        15
           7    1.00000   1.00000   1.00000         9
           8    0.54545   1.00000   0.70588         6

    accuracy                        0.91011        89
   macro avg    0.92326   0.90427   0.89227        89
weighted avg    0.93971   0.91011   0.90653        89



In [19]:
n_samples = 20

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Group 2_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Group 2_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Group 2_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples) + 1

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Group 2_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

y_4 = np.ones(n_samples) + 2

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Group 2_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

y_5 = np.ones(n_samples) + 3

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Group 2_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_6 = np.ones(n_samples) + 4

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Group 2_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

y_7 = np.ones(n_samples) + 5

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Group 2_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

y_8 = np.ones(n_samples) + 6

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Group 2_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_9 = np.ones(n_samples) + 7

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
fake_labels = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9))

fake_features = torch.Tensor(fake_features)

In [20]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[20  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  2  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0]
 [ 0  0  0 17  0  0  1  2  0]
 [ 2  1  0  0 17  0  0  0  0]
 [ 0  0  4  0  0 10  0  0  6]
 [ 0  0  0  2  0  0 18  0  0]
 [ 0  0  0  0  0  0  0 20  0]
 [ 0  0  1  0  0  1  0  0 18]]
              precision    recall  f1-score   support

         0.0    0.90909   1.00000   0.95238        20
         1.0    0.94737   0.90000   0.92308        20
         2.0    0.80000   1.00000   0.88889        20
         3.0    0.89474   0.85000   0.87179        20
         4.0    0.89474   0.85000   0.87179        20
         5.0    0.90909   0.50000   0.64516        20
         6.0    0.94737   0.90000   0.92308        20
         7.0    0.90909   1.00000   0.95238        20
         8.0    0.75000   0.90000   0.81818        20

    accuracy                        0.87778       180
   macro avg    0.88461   0.87778   0.87186       180
weighted avg    0.88461   0.87778   0.87186       180

